In [ ]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import networkx as nx
import warnings
warnings.simplefilter('ignore') # for networkx drawing
import matplotlib.pylab
%matplotlib inline
matplotlib.pylab.rcParams["figure.figsize"] = (14, 18)

## Locate the data files

In [ ]:
datadir = pathlib.Path("/home/david/pro/scc/data")

Headers are:

- 0  amt
- 1  herred
- 2  sogn
- 3  navn
- 4  køn
- 5  fødested
- 6  fødeaar
- 7  civilstand
- 8  position
- 9  erhverv
- 10 husstnr
- 11 kipnr
- 12 løbenr

In [ ]:
Entry = collections.namedtuple("Entry", "amt,herred,sogn,navn,køn,fødested,fødeår,civilstand,position,erhverv,husstnr,kipnr,løbenr")

In [ ]:
list(datadir.glob("lc_*.csv"))

## Look at which areas to filter on

In [ ]:
areas = {}

In [ ]:
for fn in sorted(datadir.glob("lc_*.csv")):
    print(fn.stem)
    with fn.open("r", encoding="utf-8") as fd:
        next(fd)
        areas[fn.stem] = collections.Counter(line.split("|")[1] for line in fd)

In [ ]:
areas

In [ ]:
del(areas["lc_FT1885_SDU"]) # only copenhagen, ignore for now

In [ ]:
print(functools.reduce(set.intersection, (set(arealist) for arealist in areas.values())))

In [ ]:
areas.keys()

In [ ]:
subset = []
area = "læsø"
unhandled = set()
for fnstem in areas:
    year = int(re.search(r"\d{4}", fnstem).group(0))
    print(year)
    fn = datadir / (fnstem + ".csv")
    with fn.open("r", encoding="UTF-8") as fd:
        next(fd)
        acc = []
        for line in fd:
            row = line.strip().split("|")
            if not (row[1] == area and row[3].startswith("an")):
                continue
            # fødeår som int
            row[6] = int(row[6])
            # prøv at fikse fødesteder
            fødested = row[5].replace(".","").split()
            if len(fødested) > 1:
                if fødested[-2:] == ["i", "sognet"] or fødested == ["heri", "sognet"] or "her i s" in row[5]: # {her ,}i sognet
                    fødested = row[2] # erstat med sognet
                elif "sogn" in fødested: # ... sogn på læsø / ... sogn ... amt
                    index = fødested.index("sogn")
                    fødested = " ".join(fødested[:index])
                elif "amt" in fødested:
                    index = fødested.index("amt") # ... by amt
                    fødested = " ".join(fødested[:index-1])
                elif fødested[-1].startswith("["): # ... [by]
                    fødested = " ".join(fødested[:-1])
                else:
                    fødested = " ".join(fødested)
                    unhandled.add(fødested)
                #    print(fødested)
                row[5] = fødested
            row = Entry(*row)
            acc.append(row)
        subset.append((year, acc))

In [ ]:
print([(year, len(data), len(set(data))) for (year, data) in subset])

In [ ]:
len(set(row[3] for row in subset[0][1]))

In [ ]:
(a_year, a_data), (b_year, b_data) = subset[:2]

In [ ]:
diff_name = difflib.SequenceMatcher()
diff_place = difflib.SequenceMatcher()

In [ ]:
G = nx.Graph()
for i, a_row in enumerate(a_data):
    diff_name.set_seq1(a_row.navn)
    diff_place.set_seq1(a_row.fødested)

    for j, b_row in enumerate(b_data):
        # first filter really bad matches
        age_diff = abs(a_row.fødeår - b_row.fødeår)
        if age_diff > 3:
            continue

        diff_name.set_seq2(b_row.navn)
        ratio_name = diff_name.ratio()
        if ratio_name < 0.85:
            continue
        
        diff_place.set_seq2(b_row.fødested)
        ratio_place = diff_place.ratio()
        
        # if maybe decent match, add edge
        w = ratio_name**2 * ratio_place * 1/(1+age_diff/3)
        G.add_edge("a" + str(i), "b" + str(j), weight=w)

In [ ]:
pos = {}
A = set()
B = set()
for n in G.nodes_iter():
    if n.startswith("a"):
        pos[n] = (0, int(n[1:]))
        A.add(n)
    else:
        pos[n] = (4, int(n[1:]))
        B.add(n)

In [ ]:
for k,v in sorted(nx.degree(G, A).items(), key=operator.itemgetter(1), reverse=True):
    print(k, v)

In [ ]:
match = nx.max_weight_matching(G, maxcardinality=True)
match = set(tuple(sorted(pair)) for pair in match.items())

In [ ]:
dummy = nx.Graph()
for u, v in match:
    dummy.add_edge(u,v)
nx.draw(G, pos=pos, width=0.4, node_size=5)
nx.draw_networkx_edges(dummy, edge_color="red", pos=pos, width=0.6)

In [ ]:
matched_pairs = []
for u, v in match:
    a_row = a_data[int(u[1:])]
    b_row = b_data[int(v[1:])]
    matched_pairs.append((G[u][v]["weight"], a_row, b_row))
matched_pairs.sort(reverse=True)

for (w, a, b) in matched_pairs:
    print(round(w, 2))
    print(a)
    print(b)
    print()

Bemærk: kvinder skifter navn, så vi bør smide efternavn væk på kvinder i matching.

Og vi skal have sogne som subsets af områder ift. fødested; ane marie knudsdatter født på læsøe eller byrum er lige godt.

HOV! Hvorfor er der over 5mio rows i nogle af filerne? Så mange mennesker var der ikke!